In [1]:
import pandas as pd
import sys
import spotipy
import spotipy.util as util
import requests
import numpy as np
import pyen
from spotipy.oauth2 import SpotifyClientCredentials
from pandas.io.json import json_normalize
import time

In [2]:
username='jkyung'
 
df = pd.read_csv("billboard_hits_2.csv", index_col = 0)

my_client_id='3b3dde550a184435a17323fce0ce1ea5'
my_client_secret='7d319263b759442fb3a604524d74c5b2'
 
scope = 'user-library-read playlist-modify-public playlist-modify-private'

In [3]:
# # get track ids; used something else instead
# token = util.prompt_for_user_token(username, scope,client_id=my_client_id,client_secret=my_client_secret,redirect_uri='http://localhost:8888/')
# myAuth="Bearer " + token
 
# notfound=pd.DataFrame(columns=['Songs', 'Artists'])
# tid = []

# if token:
#     sp = spotipy.Spotify(auth=token)
 
#     for index,row in df.iterrows():
#         song=row.Songs
#         artist=row.Artists
        
#         results = sp.search(song)
#         print(results['tracks'])
        
#         for i, t in enumerate(results['tracks']['items']):
#             if(t['artists'][0]['name']==artist):
#                 trackID = t['id']
#                 tid.append(trackID)
                

#             else:
#                 #print('Could not find song',song,'by artist',artist)
#                 tid.append(np.nan)
#                 #notfound=notfound.append({'Songs':song, 'Artists':artist},ignore_index=True)
#             break
    
# else:
#     print("Can't get token for", username)

In [4]:
token = util.prompt_for_user_token(username, scope,client_id=my_client_id,client_secret=my_client_secret,redirect_uri='http://localhost:8888/')
myAuth="Bearer " + token

sp = spotipy.Spotify(auth=token)

In [5]:
df.head()

,Year,Songs,Artists,Track_ID
0,2014,Happy,Pharrell Williams,spotify:track:6NPVjNh8Jhru9xOmyQigds
1,2014,Dark Horse,Katy Perry,spotify:track:5jrdCoLpJSvHHorevXBATy
2,2014,All of Me,John Legend,spotify:track:3U4isOIWM3VvDubwSI3y7a
3,2014,Fancy,Iggy Azalea,spotify:track:3oiMJQAWVaxSubJ7b2VUtX
4,2014,Counting Stars,OneRepublic,spotify:track:6sy3LkhNFjJWlaeSMNwQ62


In [6]:
features = []

for i,tid in enumerate(df.Track_ID):
    if i%300 == 0:
        time.sleep(10)
    feature = sp.audio_features(tid)
    features.append(feature[0])

In [7]:
features
len(features)

397

In [8]:
features_df = json_normalize(features)
features_df.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.28600,https://api.spotify.com/v1/audio-analysis/6NPV...,0.652,233305,0.757,6NPVjNh8Jhru9xOmyQigds,0.0,1,0.0886,-6.819,1,0.1530,159.911,4,https://api.spotify.com/v1/tracks/6NPVjNh8Jhru...,audio_features,spotify:track:6NPVjNh8Jhru9xOmyQigds,0.962
1,0.00403,https://api.spotify.com/v1/audio-analysis/5jrd...,0.646,215672,0.587,5jrdCoLpJSvHHorevXBATy,0.0,6,0.1450,-6.139,1,0.0528,131.932,4,https://api.spotify.com/v1/tracks/5jrdCoLpJSvH...,audio_features,spotify:track:5jrdCoLpJSvHHorevXBATy,0.341
2,0.92200,https://api.spotify.com/v1/audio-analysis/3U4i...,0.422,269560,0.264,3U4isOIWM3VvDubwSI3y7a,0.0,8,0.1320,-7.064,1,0.0322,119.930,4,https://api.spotify.com/v1/tracks/3U4isOIWM3Vv...,audio_features,spotify:track:3U4isOIWM3VvDubwSI3y7a,0.331
3,0.10200,https://api.spotify.com/v1/audio-analysis/3oiM...,0.911,199938,0.707,3oiMJQAWVaxSubJ7b2VUtX,0.0,8,0.0490,-4.136,1,0.0696,94.964,4,https://api.spotify.com/v1/tracks/3oiMJQAWVaxS...,audio_features,spotify:track:3oiMJQAWVaxSubJ7b2VUtX,0.375
4,0.06490,https://api.spotify.com/v1/audio-analysis/6sy3...,0.663,257840,0.714,6sy3LkhNFjJWlaeSMNwQ62,0.0,1,0.1160,-4.944,0,0.0380,121.990,4,https://api.spotify.com/v1/tracks/6sy3LkhNFjJW...,audio_features,spotify:track:6sy3LkhNFjJWlaeSMNwQ62,0.468


In [9]:
# combine dataframes
df_result = pd.concat([df, features_df], axis=1, join_axes=[df.index])

In [10]:
df_result.head()

,Year,Songs,Artists,Track_ID,acousticness,analysis_url,danceability,duration_ms,energy,id,...,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,2014,Happy,Pharrell Williams,spotify:track:6NPVjNh8Jhru9xOmyQigds,0.28600,https://api.spotify.com/v1/audio-analysis/6NPV...,0.652,233305.0,0.757,6NPVjNh8Jhru9xOmyQigds,...,0.0886,-6.819,1.0,0.1530,159.911,4.0,https://api.spotify.com/v1/tracks/6NPVjNh8Jhru...,audio_features,spotify:track:6NPVjNh8Jhru9xOmyQigds,0.962
1,2014,Dark Horse,Katy Perry,spotify:track:5jrdCoLpJSvHHorevXBATy,0.00403,https://api.spotify.com/v1/audio-analysis/5jrd...,0.646,215672.0,0.587,5jrdCoLpJSvHHorevXBATy,...,0.1450,-6.139,1.0,0.0528,131.932,4.0,https://api.spotify.com/v1/tracks/5jrdCoLpJSvH...,audio_features,spotify:track:5jrdCoLpJSvHHorevXBATy,0.341
2,2014,All of Me,John Legend,spotify:track:3U4isOIWM3VvDubwSI3y7a,0.92200,https://api.spotify.com/v1/audio-analysis/3U4i...,0.422,269560.0,0.264,3U4isOIWM3VvDubwSI3y7a,...,0.1320,-7.064,1.0,0.0322,119.930,4.0,https://api.spotify.com/v1/tracks/3U4isOIWM3Vv...,audio_features,spotify:track:3U4isOIWM3VvDubwSI3y7a,0.331
3,2014,Fancy,Iggy Azalea,spotify:track:3oiMJQAWVaxSubJ7b2VUtX,0.10200,https://api.spotify.com/v1/audio-analysis/3oiM...,0.911,199938.0,0.707,3oiMJQAWVaxSubJ7b2VUtX,...,0.0490,-4.136,1.0,0.0696,94.964,4.0,https://api.spotify.com/v1/tracks/3oiMJQAWVaxS...,audio_features,spotify:track:3oiMJQAWVaxSubJ7b2VUtX,0.375
4,2014,Counting Stars,OneRepublic,spotify:track:6sy3LkhNFjJWlaeSMNwQ62,0.06490,https://api.spotify.com/v1/audio-analysis/6sy3...,0.663,257840.0,0.714,6sy3LkhNFjJWlaeSMNwQ62,...,0.1160,-4.944,0.0,0.0380,121.990,4.0,https://api.spotify.com/v1/tracks/6sy3LkhNFjJW...,audio_features,spotify:track:6sy3LkhNFjJWlaeSMNwQ62,0.468


In [11]:
# filter only useful columns
df_result.drop(['analysis_url', 'id', 'track_href','uri'], inplace = True, axis=1)

In [12]:
df_result.head()

,Year,Songs,Artists,Track_ID,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,type,valence
0,2014,Happy,Pharrell Williams,spotify:track:6NPVjNh8Jhru9xOmyQigds,0.28600,0.652,233305.0,0.757,0.0,1.0,0.0886,-6.819,1.0,0.1530,159.911,4.0,audio_features,0.962
1,2014,Dark Horse,Katy Perry,spotify:track:5jrdCoLpJSvHHorevXBATy,0.00403,0.646,215672.0,0.587,0.0,6.0,0.1450,-6.139,1.0,0.0528,131.932,4.0,audio_features,0.341
2,2014,All of Me,John Legend,spotify:track:3U4isOIWM3VvDubwSI3y7a,0.92200,0.422,269560.0,0.264,0.0,8.0,0.1320,-7.064,1.0,0.0322,119.930,4.0,audio_features,0.331
3,2014,Fancy,Iggy Azalea,spotify:track:3oiMJQAWVaxSubJ7b2VUtX,0.10200,0.911,199938.0,0.707,0.0,8.0,0.0490,-4.136,1.0,0.0696,94.964,4.0,audio_features,0.375
4,2014,Counting Stars,OneRepublic,spotify:track:6sy3LkhNFjJWlaeSMNwQ62,0.06490,0.663,257840.0,0.714,0.0,1.0,0.1160,-4.944,0.0,0.0380,121.990,4.0,audio_features,0.468


In [13]:
df_result.to_csv('billboard_hits_3.csv')